# Feature Selection

In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import joblib
from sklearn import preprocessing # LabelEncoder
from sklearn.preprocessing import MinMaxScaler # Escala los datos
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import scikitplot as skplt 
from string import ascii_uppercase 
# import seaborn as sns
import qgrid
import time
#from sklearn.externals import joblib # Para guardar el modelo
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.pipeline import Pipeline as PIPELINE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from scipy import stats #Para la moda

from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [2]:
def loadModel(path):    
    clf = joblib.load(path)
    
    return clf;

In [3]:
def saveModel(clf, name):
    path = "../../Models/" + name + ".pkl"
    joblib.dump(clf, path) 

In [4]:
def loadCSV(pathSamples, pathMatrix):
    df_loaded = pd.read_table(pathMatrix, sep=',')
    data = pd.read_table(pathSamples, sep=',')
    clin_trial_values = df_loaded.values
    
    Y = data['Eligible']
    Y = Y.astype(int)
    X = clin_trial_values[:, :]
    return X, Y

In [23]:
def select_features(modelo, n_features, fwd, fltg):

    sfs = SFS(modelo, 
           k_features=n_features,
           forward=fwd,
           floating=fltg,
           verbose=1,
           scoring='f1',
           cv=0,
           n_jobs=-1)
    
    return sfs

In [12]:
def selection(features, model, fwd=True, fltg=False):        
    #Para calcular el costo computacional
    tiempo_i = time.time()

    accuracy_list = np.zeros([4,2])
    accuracy_list_train = np.zeros([4,2])
    precision_list = np.zeros([4,2])
    precision_list_train = np.zeros([4,2])
    recall_list = np.zeros([4,2])
    recall_list_train = np.zeros([4,2])
    f_list = np.zeros([4,2]) 
    f1_train = np.zeros([4,2])
    
    feature_list = [] #Para guardar los indices de las características que eligió en cada iteración
    XN_list = [] #Para guardar los diferentes X
    sf_list = []
    
    #Implemetamos la metodología de validación 
    skf = StratifiedKFold(n_splits=4)
    
    for i, indexes in enumerate(skf.split(X, Y)):
        #Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=.2) # Modificar metodología de validación
        #scaler = MinMaxScaler() #Escala entre 0 y 1
        #Xtrain = scaler.fit_transform(Xtrain)
        #Xtest = scaler.transform(Xtest)
        
        Xtrain, Xtest = X[indexes[0]], X[indexes[1]]
        Ytrain, Ytest = Y[indexes[0]], Y[indexes[1]]

        sf = select_features(model, features, fwd, fltg)
        sf =  sf.fit(Xtrain, Ytrain)

        X_train_sfs = sf.transform(Xtrain)
        X_test_sfs = sf.transform(Xtest)
        XN = sf.transform(X)

        model.fit(X_train_sfs, Ytrain)
        pred = model.predict(X_test_sfs)
        pred_train = model.predict(X_train_sfs)
        
        # accuracy
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[i] = _accuracy_

        _accuracy_train_ = accuracy_score(Ytrain, pred_train, normalize=True)
        accuracy_list_train[i] = _accuracy_train_

        # recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[i] = _recalls_
        
        _recalls_train_ = accuracy_score(Ytrain, pred_train, normalize=True)
        recall_list_train[i] = _recalls_train_

        # precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[i] = _precisions_
        
        _precisions_train_ = precision_score(Ytrain, pred_train, average=None)
        precision_list_train[i] = _precisions_train_
        
        # f1
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[i] = _f_score_
        
        _f_score_train = f1_score(Ytrain, pred_train, average=None)
        f1_train[i] = _f_score_train
        
        feature_list.extend(sf.k_feature_idx_)
        
        XN_list.append(XN)
        sf_list.append(sf)
        
    report = {
        'f1':[np.around(np.mean(f1_train), decimals=5), np.around(np.std(f1_train), decimals=5), np.around(np.mean(f_list), decimals=5), np.around(np.std(f_list), decimals=5)],
        'accuracy':[np.around(np.mean(accuracy_list_train), decimals=5), np.around(np.std(accuracy_list_train), decimals=5), np.around(np.mean(accuracy_list), decimals=5), np.around(np.std(accuracy_list), decimals=5)],
        'precision':[np.around(np.mean(precision_list_train), decimals=5), np.around(np.std(precision_list_train), decimals=5), np.around(np.mean(precision_list), decimals=5), np.around(np.std(precision_list), decimals=5)],
        'recall':[np.around(np.mean(recall_list_train), decimals=5), np.around(np.std(recall_list_train), decimals=5), np.around(np.mean(recall_list), decimals=5), np.around(np.std(recall_list), decimals=5)]
    }

    print(pd.DataFrame.from_dict(report, orient='index', columns=['Training score', '+/-', 'Test score', '+/-']))

    print("Total time:", time.time()-tiempo_i)
    
    
    index = np.where(f_list == np.amax(f_list)) #Busca cual es el resultado más grande en la lista de f1   
    idx = index[0][0] #pasa el índice de array a entero   
    XN = XN_list[idx] #Selecciona el mejor X 
    sf = sf_list[idx] #índices de las mejores características
    
    mode, count = stats.mode(feature_list) #primer retorno es un vector con las modas, el segundo son las repeticiones
    
    return mode, count, XN, sf;

In [7]:
def grid_search(clf, param_grid):
    sfs = SFS(estimator=clf,
              scoring='f1',
              n_jobs=4)

    # Now only one object in the pipeline (in fact this is not even needed anymore)
    pipe = PIPELINE([('sfs', sfs)])

    gs = GridSearchCV(estimator=pipe,
                                              param_grid=param_grid,
                                              scoring='f1',
                                              n_jobs=4,
                                              refit=True)

    gs = gs.fit(X, Y)

    for i in range(len(gs.cv_results_['params'])):
        print(gs.cv_results_['params'][i], 'test f1:', gs.cv_results_['mean_test_score'][i])

    return gs

# LSA 10k_1Col_NoCarEsp

In [ ]:
loadCSV("../../Dataset/10k_1Col_NoCarEsp_LSA.csv", "../../Tables/docsTopicsLSA1200.csv") #Cargar SCV

In [ ]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({'# de características seleccionadas' : pd.Series([200,300])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['# de características seleccionadas'], inplace=True)

for k in df_types.index:
    mode, count, XN, sf, Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = selection(k)
    print(k)
    print(sf.k_feature_idx_)
    df_types["Eficiencia"][k] = Acc
    df_types["Int_Eficiencia"][k] = IntAcc
    df_types["Sensibilidad"][k] = Sen
    df_types["Int_Sensibilidad"][k] = IntSen
    df_types["Precision"][k] = Pre
    df_types["Int_Precision"][k] = IntPre
    df_types["F-Score"][k] = f
    df_types["Int_F-Score"][k] = IntF
    df_types["Error_Prueba"][k] = error
    df_types["Int_error"][k] = stdError
    df_types["Tiempo de ejecución"][k] = tiempo
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

In [ ]:
mode, count, XN, sf, Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = selection(100)


## LSA, 10k, sin caracteres especiales, 1500 Tópicos


Latent Semantic Analysis con 10mil muestras. Uniendo las columnas "Diagnoses" e "Interventions". Eliminando todos los caracteres especiales y números de las muestras. 1500 tópicos.

In [8]:
X, Y = loadCSV("../../Dataset/10k_1Col_NoCarEsp_LSA.csv", "../../Tables/docsTopicsLSA1500.csv") #Cargar SCV

In [10]:
#param_grid = {
#    "sfs__k_features": [300, 700, 1000, 1200],
#    'sfs__forward': [True, False],
#    'sfs__floating': [True, False]
#}
#clf = SVC(C=1, kernel='rbf', gamma=.1)

#gs = grid_search(clf, param_grid)

#saveModel(gs, "SFS_SVC_LSA_1500")

In [11]:
# From Fisher Score
X = np.delete(X, np.s_[
    0, 4, 6, 12, 18, 20, 21, 22, 23, 24, 25, 26, 28, 30, 31, 32, 35, 39, 40, 41, 42, 44, 47, 48, 50, 51, 55, 59, 60, 64, 66, 69, 70, 71, 74, 75, 79, 81, 82, 86, 87, 88, 89, 91, 92, 94, 96, 97, 106, 107, 108, 110, 111, 116, 117, 118, 120, 121, 122, 124, 127, 128, 129, 132, 133, 134, 135, 136, 138, 139, 143, 144, 146, 147, 149, 151, 153, 154, 155, 156, 157, 158, 162, 164, 166, 168, 169, 175, 179, 181, 182, 183, 184, 186, 188, 190, 192, 193, 196, 197, 198, 199, 202, 203, 204, 206, 208, 209, 214, 216, 217, 219, 222, 223, 224, 228, 229, 231, 232, 233, 234, 235, 236, 239, 241, 243, 246, 248, 249, 250, 253, 255, 256, 258, 262, 264, 266, 268, 269, 273, 279, 280, 281, 282, 285, 287, 290, 297, 299, 300, 301, 304, 309, 311, 312, 314, 316, 323, 325, 326, 327, 330, 331, 332, 333, 334, 337, 338, 339, 340, 341, 342, 344, 346, 348, 349, 350, 353, 354, 355, 356, 357, 361, 365, 366, 367, 371, 372, 373, 376, 377, 378, 379, 382, 383, 385, 387, 388, 390, 394, 397, 398, 401, 402, 403, 404, 405, 406, 407, 411, 412, 413, 415, 416, 419, 420, 421, 426, 427, 428, 430, 431, 432, 433, 435, 440, 442, 443, 444, 447, 449, 450, 451, 453, 454, 455, 457, 459, 462, 463, 466, 467, 468, 470, 471, 472, 474, 478, 482, 484, 486, 488, 490, 493, 494, 495, 500, 503, 505, 506, 508, 511, 513, 515, 517, 521, 522, 523, 527, 528, 529, 531, 532, 534, 535, 539, 540, 542, 544, 545, 546, 548, 553, 555, 557, 558, 560, 562, 564, 565, 567, 568, 569, 572, 573, 575, 578, 587, 588, 590, 593, 594, 595, 596, 598, 599, 601, 608, 609, 611, 612, 613, 618, 620, 624, 626, 628, 629, 631, 633, 635, 636, 637, 638, 640, 641, 643, 645, 648, 649, 657, 658, 660, 662, 665, 670, 671, 672, 674, 679, 681, 682, 683, 684, 686, 687, 688, 690, 691, 692, 693, 695, 697, 698, 699, 702, 703, 704, 707, 708, 709, 710, 711, 716, 717, 718, 720, 723, 724, 725, 726, 731, 733, 734, 736, 742, 743, 744, 745, 749, 751, 752, 753, 755, 756, 758, 761, 762, 763, 764, 765, 767, 770, 771, 773, 779, 780, 781, 791, 792, 793, 797, 799, 803, 804, 806, 808, 809, 810, 811, 812, 813, 814, 815, 817, 818, 819, 820, 821, 824, 825, 826, 828, 830, 832, 833, 834, 835, 836, 839, 840, 841, 842, 843, 845, 847, 851, 854, 857, 859, 861, 862, 865, 867, 868, 871, 873, 874, 876, 877, 882, 885, 890, 898, 899, 900, 901, 902, 903, 904, 907, 909, 910, 913, 915, 916, 917, 922, 923, 924, 929, 932, 933, 934, 937, 946, 947, 949, 952, 953, 955, 956, 958, 959, 961, 963, 964, 965, 966, 967, 969, 971, 972, 974, 975, 977, 978, 985, 987, 988, 992, 994, 995, 996, 997, 999, 1002, 1003, 1004, 1005, 1006, 1008, 1009, 1011, 1014, 1015, 1016, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1026, 1027, 1028, 1029, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1039, 1040, 1041, 1042, 1045, 1047, 1049, 1052, 1054, 1055, 1057, 1058, 1059, 1062, 1063, 1064, 1067, 1068, 1069, 1071, 1072, 1075, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1087, 1088, 1093, 1096, 1097, 1098, 1100, 1103, 1105, 1106, 1107, 1108, 1110, 1112, 1113, 1119, 1122, 1124, 1125, 1126, 1128, 1129, 1132, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1143, 1146, 1147, 1149, 1152, 1155, 1157, 1159, 1160, 1161, 1162, 1165, 1166, 1168, 1170, 1171, 1172, 1173, 1181, 1184, 1187, 1188, 1191, 1192, 1194, 1196, 1198, 1201, 1202, 1203, 1204, 1206, 1207, 1211, 1212, 1213, 1214, 1215, 1216, 1220, 1222, 1226, 1227, 1228, 1229, 1232, 1234, 1235, 1236, 1238, 1239, 1240, 1241, 1242, 1249, 1256, 1257, 1259, 1260, 1261, 1262, 1264, 1266, 1267, 1268, 1269, 1270, 1276, 1278, 1279, 1280, 1282, 1287, 1293, 1294, 1295, 1299, 1300, 1302, 1304, 1306, 1308, 1309, 1310, 1311, 1313, 1314, 1316, 1319, 1321, 1322, 1324, 1325, 1327, 1328, 1329, 1331, 1332, 1336, 1349, 1352, 1354, 1355, 1356, 1357, 1359, 1360, 1363, 1365, 1366, 1371, 1372, 1375, 1377, 1381, 1382, 1384, 1385, 1389, 1392, 1394, 1395, 1397, 1398, 1404, 1405, 1406, 1408, 1410, 1413, 1414, 1416, 1418, 1420, 1424, 1427, 1430, 1433, 1434, 1435, 1438, 1439, 1442, 1444, 1446, 1447, 1448, 1450, 1451, 1455, 1457, 1458, 1461, 1463, 1464, 1465, 1467, 1469, 1470, 1471, 1474, 1476, 1478, 1479, 1481, 1482, 1490, 1491, 1492, 1493, 1495, 1497, 1498, 1499
], axis=1) 


#### Naives Bayes

In [19]:
clf = GaussianNB()
mode, count, XN, selected_features = selection(features=100, model=clf, fwd=True, fltg=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:    6.9s finished
Features: 1/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 1499 out of 1499 | elapsed:    3.0s finished
Features: 2/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 1101 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 1498 out of 1498 | elapsed:    4.2s finished
Features: 3/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 1025 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 1497 out of 1497 | elapsed:   

[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done 1478 out of 1478 | elapsed:   33.5s finished
Features: 23/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done 1477 out of 1477 | elapsed:   34.2s finished
Features: 24/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | ela

[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   39.1s
[Parallel(n_jobs=-1)]: Done 1461 out of 1461 | elapsed:   46.2s finished
Features: 40/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   40.1s
[Parallel(n_jobs=-1)]: Done 1460 out of 1460 | elapsed:   47.1s finished
Features: 41/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | el

Features: 57/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   31.2s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   48.5s
[Parallel(n_jobs=-1)]: Done 1443 out of 1443 | elapsed:   56.6s finished
Features: 58/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   49.2s
[Parallel(n_jobs=-1)]: Done 1442 out of 1442 | elapsed:   57.2s finished
Features: 59/100[Parallel(n_jobs=-1)]: Using bac

[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   37.9s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   59.0s
[Parallel(n_jobs=-1)]: Done 1426 out of 1426 | elapsed:  1.1min finished
Features: 75/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   37.9s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   58.9s
[Parallel(n_jobs=-1)]: Done 1425 out of 1425 | elapsed:  1.1min finished
Features: 76/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | ela

...

[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   33.5s
[Parallel(n_jobs=-1)]: Done 1469 out of 1469 | elapsed:   40.0s finished
Features: 32/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   22.2s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 1468 out of 1468 | elapsed:   40.5s finished
Features: 33/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 784 tasks      | ela

[Parallel(n_jobs=-1)]: Done 1452 out of 1452 | elapsed:  1.2min finished
Features: 49/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   36.6s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   56.6s
[Parallel(n_jobs=-1)]: Done 1451 out of 1451 | elapsed:  1.1min finished
Features: 50/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done 1450 out of 1450 | e

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   48.5s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1434 out of 1434 | elapsed:  1.4min finished
Features: 67/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   26.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   43.0s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1433 out of 1433 | elapsed:  1.2min finished
Features: 68/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | ela

[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   46.5s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1417 out of 1417 | elapsed:  1.3min finished
Features: 84/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   48.7s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1416 out of 1416 | elapsed:  1.4min finished
Features: 85/100[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | ela

           Training score      +/-  Test score      +/-
f1                0.69783  0.01272     0.66670  0.01209
accuracy          0.69810  0.00923     0.66700  0.00703
precision         0.69885  0.01578     0.66762  0.01264
recall            0.69810  0.00923     0.66700  0.03088
Total time: 21360.07130265236


In [20]:
selected_features.k_feature_idx_

(1,
 2,
 9,
 13,
 14,
 29,
 31,
 33,
 34,
 40,
 43,
 44,
 85,
 95,
 99,
 106,
 141,
 142,
 156,
 159,
 171,
 181,
 186,
 195,
 230,
 245,
 247,
 260,
 262,
 264,
 273,
 280,
 281,
 283,
 294,
 295,
 302,
 313,
 315,
 336,
 338,
 348,
 358,
 375,
 379,
 380,
 381,
 396,
 398,
 403,
 411,
 414,
 431,
 433,
 448,
 452,
 464,
 470,
 482,
 492,
 495,
 498,
 510,
 514,
 515,
 525,
 530,
 545,
 551,
 570,
 577,
 623,
 686,
 737,
 771,
 777,
 799,
 808,
 825,
 865,
 903,
 916,
 958,
 959,
 985,
 988,
 989,
 994,
 1003,
 1035,
 1053,
 1057,
 1074,
 1241,
 1341,
 1386,
 1394,
 1437,
 1446,
 1477)

In [27]:
saveModel(selected_features, "SFS_100_LSA_1500")

In [ ]:
mode_tres, count_tres, XN_tres, selected_features_tres = selection(features=300, model=clf, fwd=True, fltg=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 774 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:   18.5s finished
Features: 1/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 818 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 1484 out of 1499 | elapsed:   21.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 1499 out of 1499 | elapsed:   21.5s finished
Features: 2/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 1498 out of 1498 | elapsed:    6.5s finished
Features: 3/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks  

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done 1480 out of 1480 | elapsed:   35.9s finished
Features: 21/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done 1479 out of 1479 | elapsed:   37.1s finished
Features: 22/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | ela

[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   43.2s
[Parallel(n_jobs=-1)]: Done 1463 out of 1463 | elapsed:   50.7s finished
Features: 38/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   28.4s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   44.6s
[Parallel(n_jobs=-1)]: Done 1462 out of 1462 | elapsed:   52.9s finished
Features: 39/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | ela

[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1446 out of 1446 | elapsed:  1.2min finished
Features: 55/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   58.8s
[Parallel(n_jobs=-1)]: Done 1445 out of 1445 | elapsed:  1.1min finished
Features: 56/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   37.3s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | el

Features: 72/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   49.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1428 out of 1428 | elapsed:  2.1min finished
Features: 73/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1427 out of 1427 | elapsed:  1.9min finished
Features: 74/300[Parallel(n_jobs=-1)]: Using bac

[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   40.0s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1411 out of 1411 | elapsed:  2.0min finished
Features: 90/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   38.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1410 out of 1410 | elapsed:  2.0min finished
Features: 91/300[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | ela

In [ ]:
selected_features_tres.k_feature_idx_

In [ ]:
mode_cinco, count_cinco, XN_cinco, selected_features_cinco = selection(features=500, model=clf, fwd=True, fltg=False)

In [ ]:
mode_siete, count_siete, XN_siete, selected_features_siete = selection(features=700, model=clf, fwd=True, fltg=False)

In [ ]:
mode_mil, count_mil, XN_mil, selected_features_mil = selection(features=700, model=clf, fwd=True, fltg=False)